In [1]:
# Importe el conjunto de datos de diabetes y divídalo en entrenamiento y prueba usando scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import K2Score
import pandas as pd

from infer import inferencia

# db = load_diabetes()
# X = db.data
# y = db.target
# X_train, X_test, y_train, y_test = train_test_split(X, y)

df = pd.read_csv("train.csv", sep = ";")
test = pd.read_csv("test.csv", sep = ";")
df = df.drop(df.columns[0], axis = 1 )

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [6]:
#Importe MLFlow para registrar los experimentos, el regresor de bosques aleatorios y la métrica de error cuadrático medio
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

# defina el servidor para llevar el registro de modelos y artefactos
mlflow.set_tracking_uri('http://localhost:5000')
# registre el experimento
experiment = mlflow.set_experiment("sklearn-K2")

# Aquí se ejecuta MLflow sin especificar un nombre o id del experimento. MLflow los crea un experimento para este cuaderno por defecto y guarda las características del experimento y las métricas definidas. 
# Para ver el resultado de las corridas haga click en Experimentos en el menú izquierdo. 
with mlflow.start_run(experiment_id=experiment.experiment_id):
    # # defina los parámetros del modelo
    max_indegree = 1
    max_iter = int(1e3)
    # n_estimators = 400 
    # max_depth = 12
    # max_features = 8
    # # Cree el modelo con los parámetros definidos y entrénelo
    # rf = RandomForestRegressor(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features)
    # rf.fit(X_train, y_train)
    # # Realice predicciones de prueba
    # predictions = rf.predict(X_test)
    scoring_method = K2Score(data=df)
    esth = HillClimbSearch(data=df)
    estimated_modelh = esth.estimate(
        scoring_method=scoring_method, max_indegree=max_indegree, max_iter=max_iter
    )

    from pgmpy.models import BayesianNetwork
    modelo = BayesianNetwork(list(estimated_modelh.edges()))

    df_fit = df[list(estimated_modelh.nodes())]

    from pgmpy.estimators import MaximumLikelihoodEstimator

    emv = MaximumLikelihoodEstimator(model=modelo, data=df_fit)
    modelo.fit(data=df_fit, estimator = MaximumLikelihoodEstimator) 

    from pgmpy.inference import VariableElimination

    #Modelo de inferencia
    infer = VariableElimination(modelo)

    test_fit = test[list(estimated_modelh.nodes())]

    predictions = inferencia(modelo, test_fit)
  
    # Registre los parámetros
    mlflow.log_param("max Indegree", max_indegree)
    mlflow.log_param("max Iter", max_iter)

    # mlflow.log_param("num_trees", n_estimators)
    # mlflow.log_param("maxdepth", max_depth)
    # mlflow.log_param("max_feat", max_features)
  
    # Registre el modelo
    mlflow.sklearn.log_model(modelo, "random-forest-model")
  
    # Cree y registre la métrica de interés
    y_test = test.loc[:,"target"]
    accuracy = accuracy_score(y_test, predictions, normalize=False)
    mlflow.log_metric("accuracy", accuracy)
    print(accuracy)

2023/11/28 19:38:27 INFO mlflow.tracking.fluent: Experiment with name 'sklearn-K2' does not exist. Creating a new experiment.


  0%|          | 0/1000 [00:00<?, ?it/s]

5293
